## Etherscan Notebook 

Looking at how to derive price for tokens and gas fees by pool.  Pool example can be found [here](https://app.uniswap.org/explore/pools/ethereum/0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640)

In [9]:
import requests
import pandas as pd

In [71]:
api_key = '16FCD3FTVWC3KDK17WS5PTWRQX1E2WEYV2'
action='tokentx'
address="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
startblock=0
endblock=99999999
sort='desc'
GWEI_SCALER = 1e9

In [10]:
base_url = 'https://api.etherscan.io/api'
params = {
    'module': 'account',
    'action': action,
    'address': address,
    'startblock': startblock,
    'endblock': endblock,
    'sort': sort,
    'apikey': api_key
}

response = requests.get(base_url, params=params)
if response.status_code != 200:
    #st.error(f"API request failed with status code {response.status_code}")
    print(f"API request failed with status code {response.status_code}")

data = response.json()
if data['status'] != '1':
    #st.error(f"API returned an error: {data['result']}")
    print(f"API returned an error: {data['result']}")

df = pd.DataFrame(data['result'])

expected_columns = ['hash', 'blockNumber', 'timeStamp', 'from', 'to', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'confirmations', 'tokenSymbol', 'value', 'tokenName']

for col in expected_columns:
    if col not in df.columns:
        raise Exception(f"Expected column '{col}' is missing from the response")

The values below are pulled to create the gas fees and the price percentage features for the two models.  Between the two pools the values are used to create the total gas fees (i.e. sum of the two pools gas fees at a certain time) and percentage price difference (i.e. the ratio of the difference in price and the smaller pool price).

In [56]:
df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,21500574,1735384199,0x706a6106d21d668b23f7e708fc3db05d621477bacf20...,70,0xaade44e59b15208be9fef41226298643cd3bd9388c4f...,0x9393a68e904b1f52a081cdfcf02ec56d842bf239,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,48446799084,USDC,USDC,6,3,735771,5737624595,412794,2141880,deprecated,3
1,21500574,1735384199,0x706a6106d21d668b23f7e708fc3db05d621477bacf20...,70,0xaade44e59b15208be9fef41226298643cd3bd9388c4f...,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xfe837a3530dd566401d35befcd55582af7c4dffc,14508420428985255597,Wrapped Ether,WETH,18,3,735771,5737624595,412794,2141880,deprecated,3
2,21500574,1735384199,0x938a7d80d75b5c7dbea935b7e3472914402b6d578e36...,141,0xaade44e59b15208be9fef41226298643cd3bd9388c4f...,0x6352a56caadc4f1e25cd6c75970fa768a3304e64,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,48508210634,USDC,USDC,6,1,609886,5737624595,494280,1680525,deprecated,3
3,21500574,1735384199,0x938a7d80d75b5c7dbea935b7e3472914402b6d578e36...,141,0xaade44e59b15208be9fef41226298643cd3bd9388c4f...,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x6352a56caadc4f1e25cd6c75970fa768a3304e64,14530270984661210579,Wrapped Ether,WETH,18,1,609886,5737624595,494280,1680525,deprecated,3
4,21500572,1735384175,0x77eaf14a5bfd0be57a124cb5f12bcab4ef8040270b3e...,68,0x4bb283e4dcddb331fb9731c178e8718e0ea624b640ea...,0xf081470f5c6fbccf48cc4e5b82dd926409dcdd67,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,48432971348,USDC,USDC,6,2,705431,5783278619,431546,1339391,deprecated,5


In [57]:
df['tokenName'].unique() 

array(['USDC', 'Wrapped Ether'], dtype=object)

**Manual Example of creating the price in USD or ETH**.  <br> 

The example transaction can be found [here]( https://etherscan.io/tx/0x706a6106d21d668b23f7e708fc3db05d621477bacf202bd3f62d29e7f3772230).  Each row pulled as one element of the swap transaction (pair of two rows required for a swap). First we need to scale the token values for the swap to the token fraction (i.e. how much of each whole token) using a scaling factor which is specified in each row as 'tokenDecimal'.

In [70]:
# row to pick for the swap...used just for the example.
USDC_POSITION=0
ETH_POSITION=1

#Token Amounts
USDC_TOKENS  = int(df['value'].iloc[USDC_POSITION]) / 10**int(df['tokenDecimal'].iloc[USDC_POSITION])
ETH_TOKENS   = int(df['value'].iloc[ETH_POSITION]) / 10**int(df['tokenDecimal'].iloc[ETH_POSITION])

print(f"USDC Tokens in Swap (Tx: 0x...{df['hash'].iloc[USDC_POSITION][-4:]}): {USDC_TOKENS:.4f}")
print(f"ETH_TOKENS Tokens in Swap (Tx: 0x...{df['hash'].iloc[ETH_POSITION][-4:]}): {ETH_TOKENS:.4f}")

USDC Tokens in Swap (Tx: 0x...2230): 48446.7991
ETH_TOKENS Tokens in Swap (Tx: 0x...2230): 14.5084


**Get price per ETH**

To get an intuition for how this plays out with the pool.  To derive the price per ETH, you can device the USDC tokens by the ETH tokens to get an estimate, which marries up pretty close to market price estimates, which aggregate data from ratios like this from multiple holdings across the market.

In [62]:
print(f"Price of Ethereum in USD in this pool: ${USDC_TOKENS/ETH_TOKENS:.2f}")

Price of Ethereum in USD in this pool: $3339.22


**Get gas fees in ETH**

Gas fees for a transaction include all the 'work' done. There is a rate of fee per unit of work (i.e. gasPrice) and then there is the work done (i.e. gasUsed).  gasPrice and gasUsed is in gwei which is 1e9 of an ETH.  so to convert to eth, each value needs to be converted with the 1e9 scaling.

In [83]:
gas_price_eth_tokens_per_unit = int(df['gasPrice'].iloc[0])/GWEI_SCALER
gas_used_units = int(df['gasUsed'].iloc[0])
gas_fees_eth_tokens  = gas_price_eth_tokens_per_unit* (gas_used_units / GWEI_SCALER)
gas_fees_usdc_tokens = USDC_TOKENS / ETH_TOKENS * gas_fees_eth_tokens 
print(f"Gas Price in ETH per unit: {gas_price_eth_tokens_per_unit}")
print(f"Gas Used in GWEI units for Uniswap Transaction: {gas_used_units}")
print(f"Gas fees for this Transaction in ETH: {gas_fees_eth_tokens:.5f} (ETH)")
print(f"Gas fees for this Transaction in USDC: ${gas_fees_usdc_tokens:.2f} (USDC)")

Gas Price in ETH per unit: 5.737624595
Gas Used in GWEI units for Uniswap Transaction: 412794
Gas fees for this Transaction in ETH: 0.00237 (ETH)
Gas fees for this Transaction in USDC: $7.91 (USDC)
